In [24]:
from PIL import Image;
import numpy as np;
import scipy as sci;
import csv;
import cv2;
import matplotlib.pyplot as plt;
import pandas as pd;
import os
from coins_utils import *

   
inputDir = 'CandidateDataSet2/Processed_RAW/2cent/'
input_file = [inputDir+i for i in os.listdir(inputDir) if '.jpg' in i]
#input_file = glob.glob(inputDir+"*.jpg")
#input_file = [inputDir+"P70210-161658_Candidate_1_Aug_100_0.jpg"]  #For debugging

outputDir = 'CandidateDataSet2/resized_150_yuv_nobg_RAW/2cent/' #CHANGE IT!
brightness = [-25, 0, 25]
#brightness = [0]
ROWS = 150
COLS = 150


for kfile in input_file:
    for kbright in brightness:
        tic()
        #print("Augmenting image "+str(kfile))
        BLUR = 11
        CANNY_THRESH_1 = 10
        CANNY_THRESH_2 = 200
        MASK_DILATE_ITER = 4
        MASK_ERODE_ITER = 4
        im = Image.open(kfile) #Open image
        imarray = np.array(im)
        yuv = cv2.cvtColor(imarray, cv2.COLOR_BGR2YUV) #Convert it to yuv
        y, u, v = cv2.split(yuv)
        y = y.astype(np.int16)
        y += kbright
        y = np.maximum(np.zeros(y.shape),y)
        y = np.minimum(255*np.ones(y.shape),y)
        y = y.astype(np.uint8)
        final_yuv = cv2.merge((y, u, v))
        img_bright = cv2.cvtColor(final_yuv, cv2.COLOR_YUV2BGR)
        imarraynobg, mask_nz_count = bgSubstraction(img_bright, BLUR, CANNY_THRESH_1, 
                                                    CANNY_THRESH_2, MASK_DILATE_ITER, 
                                                    MASK_ERODE_ITER)
        while(mask_nz_count < 0.6 * imarray.shape[0]*imarray.shape[1]*imarray.shape[2] and CANNY_THRESH_2 >= 4):
            if CANNY_THRESH_2 < 20:
                CANNY_THRESH_2 -=3
            else:
                CANNY_THRESH_2 -= 10
            imarraynobg, mask_nz_count = bgSubstraction(imarray, BLUR, CANNY_THRESH_1, 
                                                        CANNY_THRESH_2, MASK_DILATE_ITER, 
                                                        MASK_ERODE_ITER)
        imarray = cv2.resize(imarraynobg, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
        #imarray = cv2.erode(imarray, None, 1) 
        
        #plt.imshow(img_bright)
        #plt.show()
        
        #fileNamek = kfile.split("\\")[1]
        fileNamek = kfile.split("/")[-1]
        output_filename = outputDir+fileNamek.split(".")[0]+"_150_Brightness_"+str(kbright)+".jpg"
        img = Image.fromarray(imarray, 'RGB')
        img.save(output_filename)
        #print(output_filename)
    #cv2.imwrite("image_processed.jpg", img)
    
    
    #for kAngle in angles:
    #    M = cv2.getRotationMatrix2D((imarray.shape[1]/2,imarray.shape[0]/2),kAngle,1)
    #    rotated_image = cv2.warpAffine(imarray,M,(imarray.shape[1],imarray.shape[0]))
    #    plt.imshow(rotated_image)
    #    plt.show()
    #    fileNamek = kfile.split("\\")[1]
    #    output_filename = outputDir+fileNamek.split(".")[0]+"_Aug_"+str(kAngle)+".jpg"
    #    img = Image.fromarray(rotated_image, 'RGB')
    #    img.save(output_filename)
print("Data augmentation with brightness completed!")    

Data augmentation with brightness completed!


# Use data_ls to create the labels!